In [ ]:
from __future__ import print_function

import numpy as np
import glob

## Boilerplate path hack to give access to full clustered_SNe package
import sys, os
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))
        
from clustered_SNe import data_dir_default
from clustered_SNe.analysis.constants import m_proton, pc, yr, M_solar, \
                                   metallicity_solar
from clustered_SNe.analysis.parse import Overview, RunSummary, \
                                         Inputs, \
                                         get_full_id_from_partial_id

from clustered_SNe.analysis.database_helpers import session, \
                                                Simulation, \
                                                Simulation_Inputs, \
                                                Simulation_Status


In [ ]:
data_dir = data_dir_default
data_dir = "../energy_restarts/"


In [ ]:
ids = [Overview(filename).id for filename in glob.glob(os.path.join(data_dir, "*_overview.dat"))]

In [ ]:
qsub_system = "PBS"

paths = glob.glob("../src/tmp_in_par/*")
for path in paths:
    os.remove(path)

paths = glob.glob("../scripts/" + qsub_system + "/tmp_restart/*")
for path in paths:
    os.remove(path)

f_delete = open("delete_script", mode="w")
f_delete.write("#!/bin/bash \n")
f_delete.write("mkdir -p _tmp_ \n")

f_copy = open("copy_script", mode="w")
f_copy.write("#!/bin/bash \n")
f_copy.write("mkdir -p ../energy_restarts \n")
    
f_submit = open("submit_script", mode="w")
f_submit.write("#!/bin/bash \n")

for id in ids:

    print("id: ", id)
    run_summary = RunSummary(data_dir=data_dir, id=id)

    bad_checkpoint_name = run_summary.first_unreasonable_energy()
    if bad_checkpoint_name is None:
        continue
    bad_checkpoint_num = int(bad_checkpoint_name.split("_")[-1].strip(".dat"))
    restart_checkpoints_needed = 100 - (bad_checkpoint_num % 100)
    print("bad_checkpoint_num: ", bad_checkpoint_num)
    print("new checkpoints_needed: ", restart_checkpoints_needed)
    
    bad_checkpoint_index = np.argmax(np.array(run_summary.filenames)==bad_checkpoint_name)
    for filename in run_summary.filenames[:bad_checkpoint_index]:
        f_delete.write("cp -p       " + os.path.basename(filename) + " _tmp_/ \n")
        f_copy.write(  "cp -p " + os.path.basename(filename) + " ../energy_restarts \n")

    f_copy.write(  "cp -p " + id + "_SNe.dat            " + " ../energy_restarts \n")
    f_copy.write(  "cp -p " + id + "_overview.dat       " + " ../energy_restarts \n")
    f_copy.write(  "cp -p " + id + "_inputs.dat         " + " ../energy_restarts \n")

    f_delete.write("rm -f       " + id + "_checkpoint* " + "\n")
    f_delete.write("cp -p _tmp_/" + id + "* . \n")
    
    restart_time_needed = run_summary.overview.SNe_times.max() * 3 - run_summary.times[bad_checkpoint_index]
    if bad_checkpoint_num > 99:
        restart_time_needed = 3e15
    print("extra time needed: ", restart_time_needed, "[s]")

    print("")

    CFL = .0125
        
    f_submit.write("qsub " +
                       "-v RESTART_ID=" + run_summary.id + ","
                       + "RESTART_N_CHECKPOINTS=" + str(restart_checkpoints_needed) + ","
                       + "RESTART_DELTA_TIME=" + str(restart_time_needed) + ","
                       + "RESTART_CFL=" + str(CFL)
                    + " -N " + id + "_restart.batch "
                    + " ../scripts/" + qsub_system + "/restart.batch.env_variables \n")
        
f_delete.write("rm -rf _tmp_ \n")
f_delete.close()
f_submit.close()
f_copy.close()

In [ ]:
for id in ids:
    session.query(Simulation_Status).filter(Simulation_Status.id == id).update({"status":"Running"})
    sim_status = session.query(Simulation_Status).get(id)
    print(sim_status.status)
    print(id)

In [ ]:
session.commit()